In [1]:
__author__ = "konwar.m"
__copyright__ = "Copyright 2022, AI R&D"
__credits__ = ["konwar.m"]
__license__ = "Individual Ownership"
__version__ = "1.0.1"
__maintainer__ = "konwar.m"
__email__ = "rickykonwar@gmail.com"
__status__ = "Development"

In [2]:
# Importing Libraries
import time
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb

from numpy import sqrt 
from sklearn.metrics import mean_squared_error

c:\Users\manash.jyoti.konwar\Documents\AI_Random_Projects\virtualEnvs\MLPricingVenv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Importing Libraries

In [3]:
os.chdir('..')
os.getcwd()

'c:\\Users\\manash.jyoti.konwar\\Documents\\AI_Random_Projects\\ML-Retail-Sales'

### Loading Training Data and Features

In [4]:
train_test_set = pd.read_csv(r'datasets\training_datasets\trainset.csv')
with open(r'datasets\training_datasets\new_features.pkl', 'rb') as feature_file:
    new_features = pickle.load(feature_file)

In [5]:
print(train_test_set.shape)
print(len(new_features))

(11128050, 81)
75


In [6]:
def validate_item_prices(**kwargs):
    train_test_data = kwargs.get('train_test_data')
    sales_data = kwargs.get('sales_data')

    break_counter = 1
    revamped_train_test_data = train_test_data.copy()
    pos_data = pd.DataFrame(columns=['']) 
    if isinstance(train_test_data, pd.DataFrame):
        for row in train_test_data.itertuples():
            if break_counter>3:
                break
            print(row)
            break_counter+=1

In [7]:
sales_data = pd.read_csv(r'datasets\sales_train.csv')
sales_data.shape

(2935849, 6)

In [8]:
train_test_set, pos_data = validate_item_prices(
                                train_test_data=train_test_set,
                                sales_data=sales_data
                            )

Pandas(Index=0, shop_id=59, item_id=22154, date_block_num=0, item_price=999.0, item_cnt_month=1.0, item_category_id=28, prev_shopitem_sales_1=0.0, prev_shopitem_sales_2=0.0, prev_shopitem_sales_3=0.0, prev_shopitem_sales_4=0.0, prev_shopitem_sales_5=0.0, prev_shopitem_sales_6=0.0, prev_shopitem_sales_7=0.0, prev_shopitem_sales_8=0.0, prev_shopitem_sales_9=0.0, prev_shopitem_sales_10=0.0, prev_shopitem_sales_11=0.0, prev_shopitem_sales_12=0.0, prev_item_sales_1=0.0, prev_item_sales_2=0.0, prev_item_sales_3=0.0, prev_item_sales_4=0.0, prev_item_sales_5=0.0, prev_item_sales_6=0.0, prev_item_sales_7=0.0, prev_item_sales_8=0.0, prev_item_sales_9=0.0, prev_item_sales_10=0.0, prev_item_sales_11=0.0, prev_item_sales_12=0.0, prev_shopitem_price_1=nan, prev_shopitem_price_2=nan, prev_shopitem_price_3=nan, prev_shopitem_price_4=nan, prev_shopitem_price_5=nan, prev_shopitem_price_6=nan, prev_shopitem_price_7=nan, prev_shopitem_price_8=nan, prev_shopitem_price_9=nan, prev_shopitem_price_10=nan, pre

TypeError: cannot unpack non-iterable NoneType object

### Preparing baseline data

In [ ]:
def prepare_data(**kwargs):
    train_test_data = kwargs.get('train_test_data')
    baseline_features = kwargs.get('baseline_features')
    
    # For training: rows having date_block_num from 0 to 31 is utilized for training the model
    train_time_range_lo = (train_test_data['date_block_num'] >= 0)
    train_time_range_hi =  (train_test_data['date_block_num'] <= 32)

    # For validation: rows having date_block_num as 33
    validation_time =  (train_test_data['date_block_num'] == 33)

    # For testing: rows having date_block_num from 34
    test_time =  (train_test_data['date_block_num'] == 34)

    # Retrieve rows for train set, val set, test set
    cv_trainset = train_test_data[train_time_range_lo & train_time_range_hi]
    cv_valset = train_test_data[validation_time]
    cv_trainset = cv_trainset[baseline_features]
    cv_valset = cv_valset[baseline_features]
    testset = train_test_data[test_time]
    testset = testset[baseline_features]

    # Prepare numpy arrays for training/val/test
    cv_trainset_vals = cv_trainset.values.astype(int)
    trainx = cv_trainset_vals[:, 0:len(baseline_features) - 1]
    trainy = cv_trainset_vals[:, len(baseline_features) - 1]

    cv_valset_vals = cv_valset.values.astype(int)
    valx = cv_valset_vals[:, 0:len(baseline_features) - 1]
    valy = cv_valset_vals[:, len(baseline_features) - 1]

    testset_vals = testset.values.astype(int)
    testx = testset_vals[:, 0:len(baseline_features) - 1]

    return trainx, trainy, valx, valy, testx

### Training XG Boost Regressor

In [ ]:
def train_xgboost(**kwargs):
    train_x = kwargs.get('train_x')
    train_y = kwargs.get('train_y')
    val_x = kwargs.get('val_x')
    val_y = kwargs.get('val_y')

    print('Fitting...')

    model = xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 4)
    model.fit(train_x, train_y, eval_metric='rmse')

    preds = model.predict(val_x)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    print('val set rmse: ', sqrt(mean_squared_error(val_y, preds)))

    preds = model.predict(testx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    df = pd.DataFrame(preds, columns = ['item_cnt_month'])
    df['ID'] = df.index
    df = df.set_index('ID')
    print('test predictions written to file')

    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val, test set')
    return model, df

In [ ]:
baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num'] +  new_features + ['item_cnt_month']
train_x_wo_price, train_y_wo_price, val_x_wo_price, val_y_wo_price, test_x_wo_price = prepare_data(
                                                                                        train_test_data=train_test_set,
                                                                                        baseline_features=baseline_features
                                                                                    )

In [ ]:
baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_price'] +  new_features + ['item_cnt_month']
train_x_w_price, train_y_w_price, val_x_w_price, val_y_w_price, test_x_w_price = prepare_data(
                                                                                        train_test_data=train_test_set,
                                                                                        baseline_features=baseline_features
                                                                                    )

### Training using XGBoost Regressor

In [ ]:
def train_xgboost():
    current = time.time()

    baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num'] +  new_features + ['item_cnt_month']

    # Clipping to range 0-20
    train_test_set['item_cnt_month'] = train_test_set.item_cnt_month.fillna(0).clip(0,20)

     # train: want rows with date_block_num from 0 to 31
    train_time_range_lo = (train_test_set['date_block_num'] >= 0)
    train_time_range_hi =  (train_test_set['date_block_num'] <= 32)

    # val: want rows with date_block_num from 22
    validation_time =  (train_test_set['date_block_num'] == 33)

    # test: want rows with date_block_num from 34
    test_time =  (train_test_set['date_block_num'] == 34)

    # Retrieve rows for train set, val set, test set
    cv_trainset = train_test_set[train_time_range_lo & train_time_range_hi]
    cv_valset = train_test_set[validation_time]
    cv_trainset = cv_trainset[baseline_features]
    cv_valset = cv_valset[baseline_features]
    testset = train_test_set[test_time]
    testset = testset[baseline_features]

    # Prepare numpy arrays for training/val/test
    cv_trainset_vals = cv_trainset.values.astype(int)
    trainx = cv_trainset_vals[:, 0:len(baseline_features) - 1]
    trainy = cv_trainset_vals[:, len(baseline_features) - 1]

    cv_valset_vals = cv_valset.values.astype(int)
    valx = cv_valset_vals[:, 0:len(baseline_features) - 1]
    valy = cv_valset_vals[:, len(baseline_features) - 1]

    testset_vals = testset.values.astype(int)
    testx = testset_vals[:, 0:len(baseline_features) - 1]

    print('Fitting...')

    model = xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 4)
    model.fit(trainx, trainy, eval_metric='rmse')

    preds = model.predict(valx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    print('val set rmse: ', sqrt(mean_squared_error(valy, preds)))

    preds = model.predict(testx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    df = pd.DataFrame(preds, columns = ['item_cnt_month'])
    df['ID'] = df.index
    df = df.set_index('ID')
    print('test predictions written to file')

    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val, test set')
    return model, df

In [ ]:
trained_model, df_test_predictions = train_xgboost()

In [ ]:
if not os.path.exists(os.path.join('models_outputs', 'model_trained_xgb.pkl')):
    with open(os.path.join('models_outputs', 'model_trained_xgb.pkl'), 'wb') as model_file:
        pickle.dump(trained_model, model_file)